In [14]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import config

### This model will be trained using a convolutional neural network, implemented using pytorch (if all goes well...)

I have decided to use a CNN as it (most likely) suits the data that is being collected by the program best

Current reading through d2l: (chapters related to this project) <br>
Chapter 5 (MLPs) 0% <br>
Chapter 7 (CNNs) 100% <br>
Chapter 8 (Modern CNNs) 0% <br>
Chapter 9 (LSTM/RNNs) 0%

## Implementation with CNN

This also includes some of the process of me learning about CNNs

In [3]:
def corr2d(X, K): 
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [4]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)
# this performs a cross-correlation on the image 3x3 image X with the 
# 2x2 kernel K

tensor([[19., 25.],
        [37., 43.]])

In [5]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
        # initialised with weight and bias as the two parameters

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [6]:
# the forward propogation function calls the corr2d function, adding the bias

In [10]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y
# alternatively, use predefined pytorch function nn.MaxPool2d()

In [8]:
# max pooling function - can also apply padding and stride 

### Actually creating the model

Images fed into the model have been changed to grayscale using openCV2, therefore there is only one channel - unlike with RGB having 3 channels.

To make feeding the model easier, I will alter my python code so that instead of images being created in a directory on my device, I will create two csv files with the format: x-coordinate, y-coordinate, image. One each for training and testing data. The training data will be split further into training and validation data (possibly use scikit-learn).

Image refers to the one dimensional numpy array that is created by getData.py

In [15]:
# splits the data training data into training and validation data, returns
# 2 dataframes

split = 0.2 # splt for validation data

def train_test_split(csv_path, split):
    df_data = pd.read_csv(csv_path)
    len_data = len(df_data)
    # calculate the validation data sample length
    valid_split = int(len_data * split)
    # calculate the training data samples length
    train_split = int(len_data - valid_split)
    training_samples = df_data.iloc[:train_split][:]
    valid_samples = df_data.iloc[-valid_split:][:]
    return training_samples, valid_samples

Images have already been rescaled to 80x60 and converted to grayscale. Coordinates will have to be rescaled.

## Implementation with LSTM/RNN

Like previously, includes stages of my learning process.
This may be the best option, as it takes into account previous states.